# Pandas Long to Wide Format Conversion

In [184]:
import pandas as pd
import numpy as np
import time

In [185]:
df = pd.read_csv("C:\\Users\\e117150\\Documents\\Machine Learning\\Route Mileage.csv")

In [186]:
df.head(15)

,dep_dt,flt_rte,miles,rte_flt_ct,seq_id
0,6/9/2019,1,813,7,1
1,6/9/2019,1,528,7,2
2,6/9/2019,1,677,7,3
3,6/9/2019,1,"1,246",7,4
4,6/9/2019,1,210,7,5
5,6/9/2019,1,554,7,6
6,6/9/2019,1,738,7,7
7,6/9/2019,2,591,6,1
8,6/9/2019,2,591,6,2
9,6/9/2019,2,153,6,3


The intent of the following exercise is to:
 - Investigate the processing speed ot two long-to-wide dataframe manipulation approaches
 - Select the quickest of the two options, and create a function for regular use
 
The data of interest is displayed in the table above.  The desired end goal is to store airline flight mileage data (miles) in wide-format by sequence ID (seq_id) for a particular route (flt_rte).  Once converted, the mileage data will be fed to a K-means clustering algorithm to determine whether patterns in route scheduling are discernable.

In [187]:
df = df[['dep_dt','flt_rte','seq_id','miles']]
for col in ['dep_dt','flt_rte','seq_id']: df[col] = df[col].astype('category')

In [188]:
ts1 = time.process_time()
df1 = df.pivot_table(index=['dep_dt','flt_rte'],columns='seq_id',values='miles',aggfunc='max',fill_value=0)
ts2 = time.process_time()
d1 = ts2-ts1

In [189]:
ts1 = time.process_time()
df2 = df.groupby(['dep_dt','flt_rte','seq_id']).agg({'miles': np.max}).unstack('seq_id').fillna(0)
ts2 = time.process_time()
d2 = ts2-ts1

In [190]:
dn = round(d1-d2,1)
dr = round(100*(1-(d2/d1)),1)
f'Groupby aggregated {dn} seconds ({dr}%) more quickly than Pivot Table'

'Groupby aggregates 33.4 seconds (51.3%) more quickly than Pivot Table'

In [191]:
def long_to_wide(df,index_vars,col_var,val):
    'index_vars: a list of names for the fields by which the data will be grouped'
    'col_var: a string of the name for the field by which the measured value will be stored in columns'
    'val: a string of the name for the measured value'
    index_vars.append(col_var)
    w = df.groupby(index_vars).agg({val: np.max}).unstack(col_var).fillna(0)
    return(w)

In [192]:
df = long_to_wide(df,['dep_dt','flt_rte'],'seq_id','miles')

In [193]:
df.head(10)

miles                                                
seq_id                 1      2      3      4    5      6    7      8  9
dep_dt    flt_rte                                                       
6/10/2019 1        1,199  1,189  1,189    770  738      0    0      0  0
          2          425    302    239    623  612  1,204  377    314  0
          3          214    401    674  1,243  369    369  358    909  0
          4          406    406    545    883  472    439  756  1,242  0
          5          721    323    323    999  532    358  369  1,136  0
          6          581  1,143  1,881    377    0      0    0      0  0
          7          404    882    351  1,076  628    896    0      0  0
          8          762  1,620    602    369  296    333  390    605  0
          9          591  1,430    791    696  406      0    0      0  0
          10         942    942    756  1,105  899      0    0      0  0